In [103]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [104]:
# Parameters
learning_rate = 0.001
learning_rate_AM = 0.001
training_epochs = 10
batch_size = 100
display_step = 1
save_images = False

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
x_for_RMSE = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes
W1_laplacian = tf.placeholder(tf.float32, [784, 300])
W3_laplacian = tf.placeholder(tf.float32, [300, 10])

# Set model weights
W1 = tf.Variable(tf.random_normal([784, 300], mean=0, stddev=1))
b1 = tf.Variable(tf.random_normal([300], mean=0, stddev = 1))

#W2 = tf.Variable(tf.random_normal([300, 100], mean=0, stddev= 1))
#b2 = tf.Variable(tf.random_normal([100], mean=0, stddev= 1))

W3 = tf.Variable(tf.zeros([300, 10]))
b3 = tf.Variable(tf.zeros([10]))

# Construct model
def dnn(x):
    hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1); #first hidden layer
    #hidden2 = tf.nn.relu(tf.matmul(hidden1, W2) + b2); #second hidden layer
    pred = tf.nn.softmax(tf.matmul(hidden1, W3) + b3) # Softmax layer outputs prediction probabilities
    return pred

updateW1 = W1.assign(tf.add(W1, W1_laplacian))
updateW3 = W3.assign(tf.add(W3, W3_laplacian))
    
'''
# Construct model LaPlacian
def dnn_laplacian(x):
    hidden1_laplacian = tf.nn.relu(tf.matmul(x, W1+W1_laplacian) + b1); #first hidden layer
    #hidden2 = tf.nn.relu(tf.matmul(hidden1, W2) + b2); #second hidden layer
    pred_laplacian = tf.nn.softmax(tf.matmul(hidden1_laplacian, W3+W1_laplacian) + b3) # Softmax layer outputs prediction probabilities
    return pred_laplacian
'''

def AM(x, y, learning_rate=learning_rate_AM):
    pred = dnn(x)
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
    gra, = tf.gradients(cost, x)
    gra = tf.stop_gradient(gra)
    adv_x = x - (learning_rate * gra)
    adv_x = tf.clip_by_value(adv_x, 0.0, 1.0)
    return adv_x
    
# Model output
pred = dnn(x)

# Minimize error using cross entropy 
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Metrics
correct_prediction = tf.equal(tf.argmax(pred, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
prob_pred_y = tf.reduce_max(pred)
RMSE = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(x, x_for_RMSE)), axis=1))
min_RMSE = tf.reduce_min(RMSE)

In [105]:
# Start training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%02d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished! \n")
    
    # Test model with original examples
    print ("Accuracy on original test samples:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels}))
    
    
    ### Activation maximization
    AM_image = AM(x, y)
    break_probability = 0.0001
    numbers = np.zeros((10,10))
    for i in range(10): numbers[i,i]=1
        
    # Step 1: Image randomly initialized
    print("Step 1: Image randomly initialized")
    store_RMSE_random = []
    for digit in range(10):
        image = np.random.rand(1,784)
        prob_old, prob_new = 0, 1
        for i in range(100000):
            image = sess.run(AM_image, {x: image, y: [numbers[digit]]})
            if i%1000 == 0:
                prob_old = prob_new
                prob_new = sess.run(prob_pred_y, {x: image, y: [numbers[digit]]})
                print("Prediction probability:", prob_new)
                if (abs(prob_old-prob_new)<break_probability): 
                    print("No change in probability, breaking the loop")
                    print("Calculating minimum RMSE of image computed against all train samples:")
                    calc_RMSE = sess.run(min_RMSE, {x: mnist.train.images, x_for_RMSE: image})
                    store_RMSE_random.append(calc_RMSE)
                    print('RMSE of digit', digit, "is", calc_RMSE)
                    print("Saving image of digit", digit)
                    plt.imsave('images/random/randomInit_'+str(digit)+'.png', image.reshape(28, 28))
                    break
    print("Final RMSE is", np.mean(np.array(store_RMSE_random)))
                    
    # Step 2: Image initialized from train dataset mean
    print("Step 2: Image initialized from train dataset mean")
    store_RMSE_mean = []
    image_mean_truth = [np.mean(mnist.train.images, axis=0)]
    for digit in range(10):
        image_mean = image_mean_truth
        prob_old, prob_new = 0, 1
        for i in range(100000):
            image_mean = sess.run(AM_image, {x: image_mean, y: [numbers[digit]]})
            if i%1000 == 0:
                prob_old = prob_new
                prob_new = sess.run(prob_pred_y, {x: image_mean, y: [numbers[digit]]})
                print("Prediction probability:", prob_new)
                if (abs(prob_old-prob_new)<break_probability): 
                    print("No change in probability, breaking the loop")
                    print("Calculating minimum RMSE of image computed against all train samples:")
                    calc_RMSE = sess.run(min_RMSE, {x: mnist.train.images, x_for_RMSE: image_mean})
                    store_RMSE_mean.append(calc_RMSE)
                    print('RMSE of digit', digit, "is", calc_RMSE)
                    print("Saving image of digit", digit)
                    plt.imsave('images/mean/meanInit_'+str(digit)+'.png', image_mean.reshape(28, 28))
                    break
    print("Final RMSE is", np.mean(np.array(store_RMSE_mean)))

    # Step 3: Image initialized from train dataset mean for each class
    print("Step 3: Image initialized from train dataset mean for each class")
    store_RMSE_mean_class = []
    for digit in range(10):
        indexes = np.where(np.where(mnist.train.labels == 1)[1] == digit) #Indexes of desired digit in training dataset
        image_mean_class = [np.mean(mnist.train.images[indexes], axis=0)]
        prob_old, prob_new = 0, 1
        for i in range(100000):
            image_mean_class = sess.run(AM_image, {x: image_mean_class, y: [numbers[digit]]})
            if i%1000 == 0:
                prob_old = prob_new
                prob_new = sess.run(prob_pred_y, {x: image_mean_class, y: [numbers[digit]]})
                print("Prediction probability:", prob_new)
                if (abs(prob_old-prob_new)<break_probability): 
                    print("No change in probability, breaking the loop")
                    print("Calculating minimum RMSE of image computed against all train samples:")
                    calc_RMSE = sess.run(min_RMSE, {x: mnist.train.images, x_for_RMSE: image_mean_class})
                    store_RMSE_mean_class.append(calc_RMSE)
                    print('RMSE of digit', digit, "is", calc_RMSE)
                    print("Saving image of digit", digit)
                    plt.imsave('images/mean_class/meanClassInit_'+str(digit)+'.png', image_mean_class.reshape(28, 28))
                    break
    print("Final RMSE is", np.mean(np.array(store_RMSE_mean_class)))    

Epoch: 01 cost= 0.572865174
Epoch: 02 cost= 0.371025438
Epoch: 03 cost= 0.334366951
Epoch: 04 cost= 0.314203779
Epoch: 05 cost= 0.300350308
Epoch: 06 cost= 0.290548672
Epoch: 07 cost= 0.282572925
Epoch: 08 cost= 0.275964022
Epoch: 09 cost= 0.270828644
Epoch: 10 cost= 0.266577676
Optimization Finished! 

Accuracy on original test samples: 0.9245
Step 1: Image randomly initialized
Prediction probability: 0.98963773
Prediction probability: 0.9881919
Prediction probability: 0.99425197
Prediction probability: 0.9961845
Prediction probability: 0.99713194
Prediction probability: 0.99769574
Prediction probability: 0.9980751
Prediction probability: 0.99834585
Prediction probability: 0.998548
Prediction probability: 0.99870574
Prediction probability: 0.99883145
Prediction probability: 0.998933
Prediction probability: 0.99901795
No change in probability, breaking the loop
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 0 is 0.5299318
Saving image of digit 0
Pre

Prediction probability: 0.99675673
Prediction probability: 0.99702543
Prediction probability: 0.99725264
Prediction probability: 0.9974475
Prediction probability: 0.9976164
Prediction probability: 0.99776435
Prediction probability: 0.997895
Prediction probability: 0.99801105
Prediction probability: 0.99811494
Prediction probability: 0.9982083
No change in probability, breaking the loop
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 1 is 0.19567272
Saving image of digit 1
Prediction probability: 0.77207744
Prediction probability: 0.9826622
Prediction probability: 0.9911874
Prediction probability: 0.9940631
Prediction probability: 0.99550396
Prediction probability: 0.9963678
Prediction probability: 0.99694985
Prediction probability: 0.99737024
Prediction probability: 0.99768794
Prediction probability: 0.99793446
Prediction probability: 0.99813277
Prediction probability: 0.99829584
Prediction probability: 0.9984327
Prediction probability: 0.99854904
Pr

Prediction probability: 0.9966671
Prediction probability: 0.9973917
Prediction probability: 0.99785244
Prediction probability: 0.9981719
Prediction probability: 0.9984066
Prediction probability: 0.9985856
Prediction probability: 0.99872583
Prediction probability: 0.9988393
Prediction probability: 0.998933
No change in probability, breaking the loop
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 3 is 0.1691014
Saving image of digit 3
Prediction probability: 0.96098995
Prediction probability: 0.9906577
Prediction probability: 0.9945147
Prediction probability: 0.9960859
Prediction probability: 0.99693537
Prediction probability: 0.9974655
Prediction probability: 0.99783427
Prediction probability: 0.9981054
Prediction probability: 0.9983139
Prediction probability: 0.9984799
Prediction probability: 0.99861526
Prediction probability: 0.9987274
Prediction probability: 0.99882215
No change in probability, breaking the loop
Calculating minimum RMSE of image c

In [98]:
### Section 4
training_epochs1 = 20
with tf.Session() as sess1:
    sess1.run(tf.global_variables_initializer())    
    # Training cycle
    for epoch in range(training_epochs1):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess1.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%02d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished! \n")
    
    # Test model with original examples
    print ("Accuracy on original test samples and untouched weights:", sess1.run(accuracy, {x: mnist.test.images, y: mnist.test.labels}))
    
    flatten_W1_W3 = np.concatenate((sess1.run(W1).flatten(),sess1.run(W3).flatten()))
    std = np.std(flatten_W1_W3)
    a = np.array([0.01,0.1,0.5,1])
    #I calculated std above, and realized that is always very near 1. For simplicity, we will approximate std to 1 to
    #keep the lambdas
    std=1
    lambdas = a*std
    AM_image = AM(x, y)
    for l in lambdas:
        lapNoise1 = np.random.laplace(scale=l, size=[784, 300])
        lapNoise3 = np.random.laplace(scale=l, size=[300, 10])
        sess1.run(updateW1, feed_dict={W1_laplacian: lapNoise1})
        sess1.run(updateW3, feed_dict={W3_laplacian: lapNoise3})
        print("Accuracy on test images using DP with lambda", l, "is", sess1.run(accuracy, {x: mnist.test.images, y: mnist.test.labels}))
        
        #Now we perform AM
        store_RMSE_mean_class_differential_privacy = []
        for digit in range(10):
            indexes = np.where(np.where(mnist.train.labels == 1)[1] == digit) #Indexes of desired digit in training dataset
            image_mean_class = [np.mean(mnist.train.images[indexes], axis=0)]
            prob_old, prob_new = 0, 1
            for i in range(10000):
                image_mean_class = sess1.run(AM_image, {x: image_mean_class, y: [numbers[digit]]})
                if i == 10000-1:
                    print("Calculating minimum RMSE of image computed against all train samples:")
                    calc_RMSE = sess1.run(min_RMSE, {x: mnist.train.images, x_for_RMSE: image_mean_class})
                    store_RMSE_mean_class_differential_privacy.append(calc_RMSE)
                    print('RMSE of digit', digit, "is", calc_RMSE)
                    print("Saving image of digit", digit)
                    plt.imsave('images/DP_images/meanClassInitDP_lambda'+str(l)+"digit"+str(digit)+'.png', image_mean_class.reshape(28, 28))
                    
        print("Final RMSE is", np.mean(np.array(store_RMSE_mean_class_differential_privacy)))    

Epoch: 01 cost= 0.564991195
Epoch: 02 cost= 0.365994075
Epoch: 03 cost= 0.330414759
Epoch: 04 cost= 0.310330007
Epoch: 05 cost= 0.297360740
Epoch: 06 cost= 0.287524693
Epoch: 07 cost= 0.280095532
Epoch: 08 cost= 0.274021722
Epoch: 09 cost= 0.268250526
Epoch: 10 cost= 0.264081924
Epoch: 11 cost= 0.259888196
Epoch: 12 cost= 0.257097375
Epoch: 13 cost= 0.253903738
Epoch: 14 cost= 0.251242252
Epoch: 15 cost= 0.248310407
Epoch: 16 cost= 0.246600948
Epoch: 17 cost= 0.244113633
Epoch: 18 cost= 0.242392528
Epoch: 19 cost= 0.240654330
Epoch: 20 cost= 0.239079048
Optimization Finished! 

Accuracy on original test samples and untouched weights: 0.926
Accuracy on test images using DP with lambda 0.01 is 0.8708
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 0 is 0.1803478
Saving image of digit 0
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 1 is 0.11118211
Saving image of digit 1
Calculating minimum RMSE of image computed ag

/anaconda3/lib/python3.6/site-packages/matplotlib/colors.py:823: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
/anaconda3/lib/python3.6/site-packages/numpy/ma/core.py:2784: UserWarning: Warning: converting a masked element to nan.
  order=order, subok=True, ndmin=ndmin)


Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 1 is inf
Saving image of digit 1
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 2 is inf
Saving image of digit 2
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 3 is inf
Saving image of digit 3
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 4 is inf
Saving image of digit 4
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 5 is inf
Saving image of digit 5
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 6 is inf
Saving image of digit 6
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 7 is inf
Saving image of digit 7
Calculating minimum RMSE of image computed against all train samples:
RMSE of digit 8 is inf
Saving image of digit 8
Calculating minimum RMSE of image computed against all train sam